In [6]:
import pandas as pd
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import openai
import os
import json
import re

from collections import Counter

In [8]:
data = pd.read_csv('./sample_data_coex_visitors_visits_20171201_20180105.csv', sep=',', skiprows = 3)
data = data[~data['addr'].isnull()]
data = data[data['addr'].str.contains('서울')]
data

FileNotFoundError: [Errno 2] No such file or directory: './sample_data_coex_visitors_visits_20171201_20180105.csv'

In [3]:
new_data = pd.DataFrame()
new_data['hash_uid'] = data['hash_uid']
# new_data['date'] = pd.to_datetime(data['local_datehour'], format='%Y-%m-%d %H').dt.date
# new_data['hour'] = pd.to_datetime(data['local_datehour'], format='%Y-%m-%d %H').dt.hour
new_data['date'] = data['local_datehour'].str.split(" ", expand = True)[0]
new_data['hour'] = data['local_datehour'].str.split(" ", expand = True)[1]
new_data['place_name']= data['placename'] + ' ' + data['tag'].fillna('')
new_data['place_name'] = new_data['place_name'].str.strip()
new_data['place_category'] = data['cat']
new_data['place_address'] = data['addr']

new_data.reset_index(inplace=True, drop=True)
new_data

NameError: name 'data' is not defined

In [ ]:
user_50plus = new_data['hash_uid'].value_counts()[new_data['hash_uid'].value_counts().values>=50].index
select_user_50plus = [user_50plus[i] for i in random.sample(range(0, len(user_50plus)), 500)]

log_number = 40

In [ ]:
select_user_50plus['hash_uid'].value_counts()

In [ ]:
cnt = 0

most_visited_date_ans = []
most_visited_cat_ans = []

most_visited_date_ans = np.asarray(most_visited_date_ans, dtype="object")
most_visited_cat_ans = np.asarray(most_visited_cat_ans, dtype="object")

for i in select_user_50plus:
    # print('----- ',cnt,' -----')
    
    user_log = new_data[new_data['hash_uid'] == i]
    user_log = user_log.drop('hash_uid', axis=1)
    user_log = user_log[-log_number:]
    user_log.reset_index(inplace=True, drop=True)
    
    # user_log_shuffle = user_log.sample(frac=1).reset_index(drop=True)
    
    # 이후 api call에서 저장된 dataframe을 통해 로그를 보낼 예정이라면 따로 저장 필요.
    user_log.to_csv(f'./data/subtasks_temporal/user_log/user_log_{cnt}.csv', index=False)
    # user_log_shuffle.to_csv(f'./data/subtasks_temporal/user_log_shuffle/user_log_shuffle_{cnt}.csv', index=False)
    

    
    visit_count_by_date = user_log['date'].value_counts()
    most_visited_date = visit_count_by_date.idxmax()

    date_visited_number = len(user_log[user_log['date'] == most_visited_date])
    # dates_number = visit_count_by_date.tolist().count(date_visited_number)
    dates_index = [i for i, e in enumerate(visit_count_by_date.tolist()) if e == date_visited_number]
   
    most_visited_date_list = []
    most_visited_date_list.append(visit_count_by_date.keys().tolist()[:len(dates_index)])
    
    # most_visited_date_ans.append(most_visited_date_list)
    # most_visited_date_ans = np.asarray(most_visited_date_ans, dtype="object")
    most_visited_date_ans = np.append(most_visited_date_ans, most_visited_date_list)



                                  
    visit_count_by_cat = user_log['place_category'].value_counts()
    most_visited_cat = visit_count_by_cat.idxmax()

    cat_visited_number = len(user_log[user_log['place_category'] == most_visited_cat])
    cats_index = [j for j, n in enumerate(visit_count_by_cat.tolist()) if n == cat_visited_number]

    most_visited_cat_list = []
    most_visited_cat_list.append(visit_count_by_cat.keys().tolist()[:len(cats_index)])
    
    # most_visited_cat_ans.append(most_visited_cat_list)
    # most_visited_cat_ans = np.asarray(most_visited_cat_ans, dtype="object")
    most_visited_cat_ans = np.append(most_visited_cat_ans, most_visited_cat_list)
    
    
    cnt += 1